In [1]:
%run common_imports.py
import spyglass.position.v1 as sgp
from spyglass.position import PositionOutput

dj.config['display.limit'] = 10**3
%matplotlib qt
plt.style.use(["seaborn-v0_8-talk"])
%load_ext autoreload
%autoreload 2
pd.options.display.max_rows = 1000

[2025-12-13 11:55:07,568][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-13 11:55:07,590][INFO]: Connected anirudh@172.16.102.154:3306


### Insert parameters

In [2]:
bodypart1 = "nose"
bodypart2 = "tailBase"

centroid_params_name = "nose_tail_no_nan"
centroid_params = {
    "centroid_method": "two_pt_centroid",
    "points": {
        "point1": f"{bodypart1}",
        "point2": f"{bodypart2}",
    },
    "interpolate": True,
    'interp_params': {'max_cm_to_interp': 15},
    'smooth': True,
    'smoothing_params': {'smoothing_duration': 0.05, 'smooth_method': 'moving_avg'},
    'max_LED_separation': 30,  
    "speed_smoothing_std_dev": 0.100,
    
}

sgp.DLCCentroidParams.insert1(
    {
        "dlc_centroid_params_name": centroid_params_name,
        "params": centroid_params,
    },
    skip_duplicates=True,
)

In [3]:
sgp.DLCOrientationParams.insert1(
    {
        'dlc_orientation_params_name': 'nose_tail_no_nan',
        'params': {'orient_method': 'red_green_orientation', 'bodypart1': f'{bodypart1}', 'bodypart2': f'{bodypart2}', 'orientation_smoothing_std_dev': 0.001}
    }, skip_duplicates = True
)

### Begin loop

In [4]:
sgc.CameraDevice()

camera_name,meters_per_pixel height / width of pixel in meters,manufacturer,model,lens,camera_id
Camera 1,0.0006917,Allied Vision,AVT Manta,Theia,0
sleep box camera,0.00211,Allied Vision,Manta,unknown,0
SleepBox_camera,0.000842,Manta,unknown,unknown,0
w-track camera,0.00211,Allied Vision,Manta,unknown2,1


In [6]:
sgc.VideoFile() & session_restrict

nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),video_file_num,camera_name,video_file_object_id the object id of the file object
wilbur20210512_.nwb,1,0,sleep box camera,10ccb3e3-a5e6-487e-afc1-336f004ba34c
wilbur20210512_.nwb,2,1,w-track camera,3c4b28bd-49ae-4479-bfe2-e12986854b63
wilbur20210512_.nwb,3,0,sleep box camera,ae55822c-f2f4-4a5e-8a40-7b1fcab1f26a
wilbur20210512_.nwb,4,1,w-track camera,cd538c7b-88bf-4194-b3a4-cb15ae119a72
wilbur20210512_.nwb,5,0,sleep box camera,1f0d9adb-a2b3-431c-baf8-82d6867b2bb8
wilbur20210512_.nwb,6,1,w-track camera,1146844d-c2e6-47b0-9494-79b6bcac218b
wilbur20210512_.nwb,7,0,sleep box camera,0de5dfe9-c504-466a-ac9f-8ebe35721bdc
wilbur20210512_.nwb,8,1,w-track camera,6b8edaec-edf3-48e5-8700-632a86bc2fa4


In [8]:
camera_name = "w-track camera"
matching_rows = (sgc.VideoFile & session_restrict) & {"camera_name": camera_name, "video_file_num": 1}
matching_rows

nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),video_file_num,camera_name,video_file_object_id the object id of the file object
wilbur20210512_.nwb,2,1,w-track camera,3c4b28bd-49ae-4479-bfe2-e12986854b63
wilbur20210512_.nwb,4,1,w-track camera,cd538c7b-88bf-4194-b3a4-cb15ae119a72
wilbur20210512_.nwb,6,1,w-track camera,1146844d-c2e6-47b0-9494-79b6bcac218b
wilbur20210512_.nwb,8,1,w-track camera,6b8edaec-edf3-48e5-8700-632a86bc2fa4


In [9]:
model_key = sgp.DLCModelSelection().fetch1("KEY")
bodyparts = [f'{bodypart1}', f'{bodypart2}']

In [10]:
smooth_interp_params = {'dlc_si_params_name': 'just_nan_gap_2cm',
 'params': {'smooth': False,
  'interpolate': False,
  'likelihood_thresh': 0.95,
  'max_cm_between_pts': 2,
  'num_inds_to_span': 20}}

sgp.DLCSmoothInterpParams.insert1(smooth_interp_params, skip_duplicates = True)
sgp.DLCSmoothInterpParams()

dlc_si_params_name name for this set of parameters,params dictionary of parameters
default,=BLOB=
default_interp_correction,=BLOB=
default_interp_workaround,=BLOB=
just_nan,=BLOB=
just_nan_gap_2cm,=BLOB=


In [18]:
for row in matching_rows:
    col1val = row["nwb_file_name"]
    if "wilbur20210512" in col1val:  # *** change depending on rat/day!!!
        col2val = row["epoch"]
        col3val = row["video_file_num"]

        ##insert pose estimation task
        key = {"nwb_file_name": nwb_copy_file_name,
               "epoch": col2val,
               "video_file_num": col3val,
               **model_key,
               }
        
        dlc_pose_estimation = sgp.DLCPoseEstimationSelection()  # Create an instance

        pose_estimation_key = dlc_pose_estimation.insert_estimation_task(
            key,
            task_mode="trigger", #trigger or load
            params={"gputouse": 0, "videotype": "mp4"},
            )

        ##populate DLC Pose Estimation
        sgp.DLCPoseEstimation().populate(pose_estimation_key)

        #start smooth interpolation
        si_params_name = "just_nan_gap_2cm"
        si_key = pose_estimation_key.copy()
        fields = list(sgp.DLCSmoothInterpSelection.fetch().dtype.fields.keys())
        si_key = {key: val for key, val in si_key.items() if key in fields}
        # bodyparts = 
        sgp.DLCSmoothInterpSelection.insert(
            [
                {
                    **si_key,
                    "bodypart": bodypart,
                    "dlc_si_params_name": si_params_name,
                }
                for bodypart in bodyparts
            ],
            skip_duplicates=True,
        )
        
        #-----------------
        sgp.DLCSmoothInterp().populate({**si_key, "dlc_si_params_name": "just_nan_gap_2cm"})
        #-----------------
        
        
        (
            sgp.DLCSmoothInterp() & {**si_key, "bodypart": bodyparts[0]}
        ).fetch1_dataframe().plot.scatter(x="x", y="y", s=1, figsize=(5, 5))

        #smoothinterpcohort
        cohort_key = si_key.copy()
        if "bodypart" in cohort_key:
            del cohort_key["bodypart"]
        if "dlc_si_params_name" in cohort_key:
            del cohort_key["dlc_si_params_name"]
        cohort_key["dlc_si_cohort_selection_name"] = centroid_params_name
        cohort_key["bodyparts_params_dict"] = {
            f"{bodypart1}": si_params_name,
            f"{bodypart2}": si_params_name,
        }
        sgp.DLCSmoothInterpCohortSelection().insert1(
            cohort_key, skip_duplicates=True
        )
        sgp.DLCSmoothInterpCohort.populate(cohort_key)

        ##DLC Centroid
        centroid_params_name = centroid_params_name
        centroid_key = cohort_key.copy()
        fields = list(sgp.DLCCentroidSelection.fetch().dtype.fields.keys())
        centroid_key = {
            key: val for key, val in centroid_key.items() if key in fields
        }
        centroid_key["dlc_centroid_params_name"] = centroid_params_name
        sgp.DLCCentroidSelection.insert1(centroid_key, skip_duplicates=True)
        
        #-----------------
        sgp.DLCCentroid.populate(centroid_key)
        #-----------------
        
        (sgp.DLCCentroid() & centroid_key).fetch1_dataframe().plot.scatter(
            x="position_x",
            y="position_y",
            c="speed",
            colormap="viridis",
            vmin = 0,
            vmax = 100,
            alpha=0.5,
            s=0.8,
            figsize=(10, 10),
        )

        ##DLC Orientation
        dlc_orientation_params_name = centroid_params_name
        fields = list(sgp.DLCOrientationSelection.fetch().dtype.fields.keys())
        orient_key = {
            key: val for key, val in cohort_key.items() if key in fields
        }
        orient_key["dlc_orientation_params_name"] = dlc_orientation_params_name
        sgp.DLCOrientationSelection().insert1(orient_key, skip_duplicates=True)
        sgp.DLCOrientation().populate(orient_key)

        ##DLCPosV1
        fields = list(sgp.DLCPosV1.fetch().dtype.fields.keys())
        dlc_key = {
            key: val for key, val in centroid_key.items() if key in fields
        }
        dlc_key["dlc_si_cohort_centroid"] = centroid_key[
            "dlc_si_cohort_selection_name"
        ]
        dlc_key["dlc_si_cohort_orientation"] = orient_key[
            "dlc_si_cohort_selection_name"
        ]
        dlc_key["dlc_orientation_params_name"] = orient_key[
            "dlc_orientation_params_name"
        ]
        sgp.DLCPosSelection().insert1(dlc_key, skip_duplicates=True)
        sgp.DLCPosV1().populate(dlc_key)

    else:
        continue

INFO:spyglass:Pose Estimation Selection
INFO:spyglass:video_dir: /media/labuser/NA_1_2025/spyglass/wilbur/video/
INFO:spyglass:/media/labuser/NA_1_2025/spyglass/wilbur/deeplabcut/video/20210512_wilbur_02_wtrack.mp4 already exists, skipping conversion
[17:55:03][INFO] Spyglass: inserted entry into Pose Estimation Selection
INFO:spyglass:inserted entry into Pose Estimation Selection
INFO:spyglass:Pose Estimation Selection
INFO:spyglass:video_dir: /media/labuser/NA_1_2025/spyglass/wilbur/video/
INFO:spyglass:/media/labuser/NA_1_2025/spyglass/wilbur/deeplabcut/video/20210512_wilbur_04_wtrack.mp4 already exists, skipping conversion
[17:55:56][INFO] Spyglass: inserted entry into Pose Estimation Selection
INFO:spyglass:inserted entry into Pose Estimation Selection
[17:55:56][INFO] Spyglass: ----------------------
INFO:spyglass:----------------------
[17:55:56][INFO] Spyglass: Pose Estimation
INFO:spyglass:Pose Estimation
/home/labuser/miniforge3/envs/spyglass-dlc/lib/python3.10/site-packages/

Using snapshot-1030000 for model /home/labuser/Data/spyglass/J16/position_models/WtrackSep5-trainset95shuffle1/dlc-models/iteration-0/WtrackSep5-trainset95shuffle1


2025-12-13 17:55:58.505464: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-13 17:55:58.505898: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-13 17:55:58.506093: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Starting to analyze %  /media/labuser/NA_1_2025/spyglass/wilbur/deeplabcut/video/20210512_wilbur_04_wtrack.mp4
Loading  /media/labuser/NA_1_2025/spyglass/wilbur/deeplabcut/video/20210512_wilbur_04_wtrack.mp4
Duration of video [s]:  897.2 , recorded with  69.0 fps!
Overall # of frames:  61904  found with (before cropping) frame dimensions:  692 640
Starting to extract posture


100%|██████████| 61904/61904 [1:07:16<00:00, 15.34it/s]
/home/labuser/miniforge3/envs/spyglass-dlc/lib/python3.10/site-packages/deeplabcut/utils/auxiliaryfunctions.py:403: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  DataMachine.to_hdf(dataname, "df_with_missing", format="table", mode="w")
[19:03:15][WARNING] Spyglass: DEPRECATION scheduled for version 0.6: dlc_reader: PoseEstimation
[19:03:15][INFO] Spyglass: getting raw position
INFO:spyglass:getting raw position


...

2025-12-13 21:47:20.394381: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-13 21:47:20.394992: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-13 21:47:20.395231: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Starting to analyze %  /media/labuser/NA_1_2025/spyglass/wilbur/deeplabcut/video/20210512_wilbur_08_wtrack.mp4
Loading  /media/labuser/NA_1_2025/spyglass/wilbur/deeplabcut/video/20210512_wilbur_08_wtrack.mp4
Duration of video [s]:  899.75 , recorded with  69.0 fps!
Overall # of frames:  62080  found with (before cropping) frame dimensions:  692 640
Starting to extract posture


100%|██████████| 62080/62080 [1:11:55<00:00, 14.39it/s]
/home/labuser/miniforge3/envs/spyglass-dlc/lib/python3.10/site-packages/deeplabcut/utils/auxiliaryfunctions.py:403: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  DataMachine.to_hdf(dataname, "df_with_missing", format="table", mode="w")
[22:59:16][WARNING] Spyglass: DEPRECATION scheduled for version 0.6: dlc_reader: PoseEstimation
[22:59:16][INFO] Spyglass: getting raw position
INFO:spyglass:getting raw position
[22:59:17][ERROR] Spyglass: NO POS INTERVALS FOR {'nwb_file_name': 'wilbur20210512_.nwb', 'interval_list_name': '08_wtrack'};
CANNOT POPULATE PositionIntervalMap
ERROR:spyglass:NO POS INTERVALS FOR {'nwb_file_name': 'wilbur20210512_.nwb', 'interval_list_name': '08_wtrack'};
CANNOT POPULATE PositionIntervalMap
[22:59:17][INFO] Spyglass: No position intervals found for {'nwb_file_name': 'wilbur20210512_.nwb', 'epoch': 8, 'interval_list_nam

Saving results in /media/labuser/NA_1_2025/spyglass/wilbur/deeplabcut/output/wilbur20210512/wilbur20210512_08_model_WtrackSep5-trainset95shuffle1model...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


[23:00:14][INFO] Spyglass: Inserting wilbur20210512_.nwb, epoch 08 into DLCPoseEsimation
INFO:spyglass:Inserting wilbur20210512_.nwb, epoch 08 into DLCPoseEsimation
[23:00:15][INFO] Spyglass: converting to cm
INFO:spyglass:converting to cm
[23:00:15][INFO] Spyglass: adding timestamps to DataFrame
INFO:spyglass:adding timestamps to DataFrame
[23:01:12][INFO] Spyglass: Writing new NWB file wilbur20210512_YC180IJVTH.nwb
INFO:spyglass:Writing new NWB file wilbur20210512_YC180IJVTH.nwb
[23:02:54][INFO] Spyglass: converting to cm
INFO:spyglass:converting to cm
[23:02:54][INFO] Spyglass: adding timestamps to DataFrame
INFO:spyglass:adding timestamps to DataFrame
[23:03:48][INFO] Spyglass: Writing new NWB file wilbur20210512_KP0QXE94D0.nwb
INFO:spyglass:Writing new NWB file wilbur20210512_KP0QXE94D0.nwb
[23:05:26][INFO] Spyglass: converting to cm
INFO:spyglass:converting to cm
[23:05:26][INFO] Spyglass: adding timestamps to DataFrame
INFO:spyglass:adding timestamps to DataFrame
[23:06:16][INFO